In [ ]:
import asyncio
import json

from datasets import load_dataset

from tplexity.llm_client import get_llm

# login("your_huggingface_token_here")  # Используйте переменную окружения HUGGINGFACE_HUB_TOKEN

In [2]:
dataset = load_dataset("Linq-AI-Research/FinDER")["train"]

In [3]:
filtered_dataset = [row for row in dataset if len(row.get("references", [])) >= 3]
print(f"Записей с references >= 3: {len(filtered_dataset)}")

Записей с references >= 3: 23


In [4]:
llm = get_llm("qwen")


async def translate_text(text):
    messages = [
        {
            "role": "system",
            "content": "Ты переводчик. Переведи текст с английского на русский язык. Верни ТОЛЬКО перевод, без комментариев, пояснений и дополнительного текста.",
        },
        {"role": "user", "content": f"Переведи:\n\n{text}"},
    ]
    return (await llm.generate(messages, temperature=0.3, max_tokens=2000)).strip()


async def translate_record(record):
    result = {
        "text": await translate_text(record["text"]),
        "references": [await translate_text(ref) for ref in record["references"]],
        "answer": await translate_text(record["answer"]),
    }
    return result

In [5]:
translated_dataset = await asyncio.gather(*[translate_record(record) for record in filtered_dataset])

In [6]:
with open("finder_dataset.json", "w", encoding="utf-8") as f:
    json.dump(translated_dataset, f, ensure_ascii=False, indent=2)